In [ ]:
import xarray as xr
import matplotlib.pylab as plt
import Rbeast as rb
import numpy as np

# import cartopy.crs as ccrss
# import cartopy.feature as cf
import matplotlib as mpl
import pandas as pd
import my_funs
from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import BallTree, DistanceMetric

# from causalimpact import CausalImpact
import pickle
from sklearn.impute import IterativeImputer
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from scipy import stats
from collections import Counter
import seaborn as sns
import geopandas as gpd
from pyproj import CRS

mpl.rcParams["mathtext.default"] = "regular"
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from my_funs import outliers_index


def year_detect(x):
    # Get the index of the first non-nan value
    if not np.any(x):
        return np.nan
    idx = np.argmax(x)
    return idx


def most_common_values(arr, n):
    # Count the n most repeated values in an array
    counter = Counter(arr)
    most_common = counter.most_common(n)
    return [value for value, _ in most_common]


dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"

## Read the data


In [ ]:
percent = (
    xr.open_dataset("../data/percent_cover.nc")["__xarray_dataarray_variable__"].sel(
        time=slice(1984, 2013)
    )
    * 100
)
ndvi = (
    xr.open_dataarray("../data/NDVI_resampled_v2.nc").sel(time=slice("1984", "2013"))
    / 10000
)
ndvi = ndvi.drop("band")
# lai = xr.open_dataarray("../data/lai4g_annual_max.nc").sel(time=slice("1984", "2013"))
percent["lat"] = ndvi["lat"]
percent["lon"] = ndvi["lon"]
# Only focus on vegetative classes
lc = percent.isel(band=[0, 2, 3, 4])  # EF, srub, herb and barren
t_short = pd.date_range(start="1984", end="2014", freq="A-Dec").year
t = pd.date_range(start="1984", end="2014", freq="A-Dec").year
arr = xr.open_dataarray("../data/arr_id.nc")
# fires_1994 = gpd.read_file(
#     "../../data/shp_files/nbac_1986_to_2021_20220624/fire_1994_dissolved.shp"
# )
# fires_1994 = fires[fires["YEAR"] == 1994]

In [ ]:
fires = gpd.read_file(
    "../../data/shp_files/nbac_1986_to_2021_20220624/nbac_1986_to_2021_20220624.shp"
)
fires_1994 = fires[fires["YEAR"] == 1994]
fires_1994_crs = fires_1994.crs
target_crs = CRS.from_epsg(4326)
fires_1994_reprojected = fires_1994.to_crs(target_crs)

In [ ]:
fires_1994

## Distrubance and vegetation dynamic


### LCC map where at least one of the land covers changed more than 10% compared to the previous year.


In [ ]:
lc_diff = lc.diff("time")
net_change = lc_diff.where(lc_diff > 0.0).sum("band")
changed = (net_change >= 10).any(dim="time")
not_changed = (net_change < 10).all(dim="time")
mask_tmp = np.isfinite(lc).all(["band", "time"])
not_changed = not_changed.where(mask_tmp == True)
lc_changed = lc.where(changed == True)
lc_not_changed = lc.where(not_changed == True)
lc_changed_df = lc_changed.mean(["lat", "lon"]).to_pandas()

### Fit the Rbeast time series model to each pixel's percent cover from 1984 to 2013 to get the non-linear trend


In [ ]:
ef_changed = lc_changed.isel(band=0)
sh_changed = lc_changed.isel(band=1)
hb_changed = lc_changed.isel(band=2)
sparse_changed = lc_changed.isel(band=3)

metadata = rb.args(whichDimIsTime=1, season="none", startTime=1984)
prior = rb.args(trendMinSepDist=1)
mcmc = rb.args(seed=1)
extra = rb.args(  # a set of options to specify the outputs or computational configurations
    dumpInputData=True,  # make a copy of the aggregated input data in the beast ouput
    numThreadsPerCPU=1,  # Paralell  computing: use 2 threads per cpu core
    numParThreads=0,  # `0` means using all CPU cores: total num of ParThreads = numThreadsPerCPU * core Num
    printOptions=False,
    computeTrendSlope=True,
    computeCredible=True,
)
ef_changed_rbeast = rb.beast123(ef_changed.values, metadata, prior, mcmc, extra)
sh_changed_rbeast = rb.beast123(sh_changed.values, metadata, prior, mcmc, extra)
hb_changed_rbeast = rb.beast123(hb_changed.values, metadata, prior, mcmc, extra)
sparse_changed_rbeast = rb.beast123(sparse_changed.values, metadata, prior, mcmc, extra)

#### Save the results


In [ ]:
# Save the results
with open("../manuscript/outputs/ef_changed_rbeast", "wb") as fp:
    pickle.dump(ef_changed_rbeast, fp)
with open("../manuscript/outputs/sh_changed_rbeast", "wb") as fp:
    pickle.dump(sh_changed_rbeast, fp)
with open("../manuscript/outputs/hb_changed_rbeast", "wb") as fp:
    pickle.dump(hb_changed_rbeast, fp)
with open("../manuscript/outputs/sparse_changed_rbeast", "wb") as fp:
    pickle.dump(sparse_changed_rbeast, fp)

Extract the trend and some other information from fitted models for plotting.


In [ ]:
Y_ef = ef_changed_rbeast.trend.Y.reshape(30, -1)
Y_ef_mean = pd.DataFrame(Y_ef, index=t).dropna(axis=1).mean(axis=1)
Y_ef_std = pd.DataFrame(Y_ef, index=t).dropna(axis=1).std(axis=1)
cpOccPr_ef = (
    ef_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_ef = (
    ef_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_ef = (
    ef_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_ef = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_ef)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_ef_0 = ef_changed_rbeast.trend.CI[:, 0, :, :]
ci_ef_1 = ef_changed_rbeast.trend.CI[:, 1, :, :]
ci_ef_0_pd = ndvi.copy(data=ci_ef_0).mean(["lat", "lon"])
ci_ef_1_pd = ndvi.copy(data=ci_ef_1).mean(["lat", "lon"])
most_common_cp_ef = most_common_values(cp_ef.values.squeeze(), 2)

Y_sh = sh_changed_rbeast.trend.Y.reshape(30, -1)
Y_sh_mean = pd.DataFrame(Y_sh, index=t).dropna(axis=1).mean(axis=1)
Y_sh_std = pd.DataFrame(Y_sh, index=t).dropna(axis=1).std(axis=1)
cpOccPr_sh = (
    sh_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_sh = (
    sh_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_sh = (
    sh_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_sh = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_sh)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_sh_0 = sh_changed_rbeast.trend.CI[:, 0, :, :]
ci_sh_1 = sh_changed_rbeast.trend.CI[:, 1, :, :]
ci_sh_0_pd = ndvi.copy(data=ci_sh_0).mean(["lat", "lon"])
ci_sh_1_pd = ndvi.copy(data=ci_sh_1).mean(["lat", "lon"])
most_common_cp_sh = most_common_values(cp_sh.values.squeeze(), 2)

Y_hb = hb_changed_rbeast.trend.Y.reshape(30, -1)
Y_hb_mean = pd.DataFrame(Y_hb, index=t).dropna(axis=1).mean(axis=1)
Y_hb_std = pd.DataFrame(Y_hb, index=t).dropna(axis=1).std(axis=1)
cpOccPr_hb = (
    hb_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_hb = (
    hb_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_hb = (
    hb_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_hb = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_hb)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_hb_0 = hb_changed_rbeast.trend.CI[:, 0, :, :]
ci_hb_1 = hb_changed_rbeast.trend.CI[:, 1, :, :]
ci_hb_0_pd = ndvi.copy(data=ci_hb_0).mean(
    ["lat", "lon"]
)  # Lets increse the size of the figured.cp  # Detected change points with probability more than 50%
ci_hb_1_pd = ndvi.copy(data=ci_hb_1).mean(["lat", "lon"])

Y_sparse = sparse_changed_rbeast.trend.Y.reshape(30, -1)
Y_sparse_mean = pd.DataFrame(Y_sparse, index=t).dropna(axis=1).mean(axis=1)
Y_sparse_std = pd.DataFrame(Y_sparse, index=t).dropna(axis=1).std(axis=1)
cpOccPr_sparse = (
    sparse_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_sparse = (
    sparse_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_sparse = (
    sparse_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_sparse = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_sparse)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_sparse_0 = sparse_changed_rbeast.trend.CI[:, 0, :, :]
ci_sparse_1 = sparse_changed_rbeast.trend.CI[:, 1, :, :]
ci_sparse_0_pd = ndvi.copy(data=ci_sparse_0).mean(["lat", "lon"])
ci_sparse_1_pd = ndvi.copy(data=ci_sparse_1).mean(["lat", "lon"])
most_common_cp_sparse = most_common_values(cp_sparse.values.squeeze(), 2)

### Get the NDVI of disturbed regions and fit the Rbeast


In [ ]:
ndvi_changed = ndvi.where(changed == True)
ndvi_not_changed = ndvi.where(not_changed == True)
I = np.isfinite(ndvi_changed).sum(dim="time") > 25
ndvi_changed_ok = ndvi_changed.where(I == True)

In [ ]:
metadata = rb.args(whichDimIsTime=1, season="none", startTime=1984)
prior = rb.args(trendMinSepDist=1)
mcmc = rb.args(seed=1)
extra = rb.args(  # a set of options to specify the outputs or computational configurations
    dumpInputData=True,  # make a copy of the aggregated input data in the beast ouput
    numThreadsPerCPU=1,  # Paralell  computing: use 2 threads per cpu core
    numParThreads=0,  # `0` means using all CPU cores: total num of ParThreads = numThreadsPerCPU * core Num
    printOptions=False,
    computeTrendSlope=True,
    computeCredible=True,
)
ndvi_changed_rbeast = rb.beast123(ndvi_changed_ok.values, metadata, prior, mcmc, extra)

### Extract some information from the fitted ndvi model.


In [ ]:
Y_ndvi = ndvi_changed_rbeast.trend.Y.reshape(30, -1)
y_ndvi_mean = pd.DataFrame(Y_ndvi, index=t).dropna(axis=1).mean(axis=1)
y_ndvi_std = pd.DataFrame(Y_ndvi, index=t).dropna(axis=1).std(axis=1)

slp_ndvi = ndvi_changed_rbeast.trend.slp  # Vector of estimate slopes at each t
cpOccPr_ndvi = (
    ndvi_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
slpSgnPosPr_ndvi = ndvi_changed_rbeast.trend.slpSgnPosPr  # Sign of the slope
cpPr_ndvi = (
    ndvi_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
# cp_ndvi = ndvi_changed_rbeast.trend.cp[
#     cpPr_ndvi > 0.5]  # Detected change points with probability more than 50%
cp_ndvi = (
    ndvi_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
# cpPr_ndvi = cpPr_ndvi[cpPr_ndvi > 0.5]
ci_ndvi_0 = ndvi_changed_rbeast.trend.CI[:, 0, :, :]
ci_ndvi_1 = ndvi_changed_rbeast.trend.CI[:, 1, :, :]

cp_ndvi = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_ndvi)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
slp_pos_mean = (
    ndvi.copy(data=ndvi_changed_rbeast.trend.slpSgnPosPr).mean(["lat", "lon"]).values
)
slp_pos_std = (
    ndvi.copy(data=ndvi_changed_rbeast.trend.slpSgnPosPr).std(["lat", "lon"]).values
)
slp_zero_mean = (
    ndvi.copy(data=ndvi_changed_rbeast.trend.slpSgnZeroPr).mean(["lat", "lon"]).values
)
slp_neg_mean = 1 - (slp_pos_mean + slp_zero_mean)

slp_ndvi_sd = (
    ndvi.copy(data=ndvi_changed_rbeast.trend.slpSD).mean(["lat", "lon"]).values
)
most_common_years = most_common_values(cp_ndvi.values.squeeze(), 2)
slp_ndvi_pd = ndvi.copy(data=slp_ndvi).mean(["lat", "lon"])
ci_ndvi_0_pd = ndvi.copy(data=ci_ndvi_0).mean(["lat", "lon"])
ci_ndvi_1_pd = ndvi.copy(data=ci_ndvi_1).mean(["lat", "lon"])

In [ ]:
ndvi_changed_mean_rbeast = rb.beast(
    ndvi_changed_ok_mean, season="none", start=1984, tseg_minlength=1, ci=True
)

In [ ]:
y_ndvi_mean = ndvi_changed_mean_rbeast.trend.Y
ci_ndvi_0_pd = ndvi_changed_mean_rbeast.trend.CI[:, 0]
ci_ndvi_1_pd = ndvi_changed_mean_rbeast.trend.CI[:, 1]
cp_ndvi = ndvi_changed_mean_rbeast.trend.cp
most_common_years = most_common_values(cp_ndvi, 2)

In [ ]:
fig, ax = plt.subplots(
    3, 1, figsize=(6, 8), gridspec_kw={"wspace": 0.1, "hspace": 0.05}
)
ax[0].plot(t, y_ndvi_mean, "-*", color="green", alpha=0.7, label="EF")
ax[0].fill_between(t, ci_ndvi_0_pd, ci_ndvi_1_pd, color="green", alpha=0.2)
# ax[0].set_ylabel("NDVI", fontsize=14)
ax[0].tick_params(axis="y", colors="green")
ax[0].tick_params(axis="both", which="major", labelsize=14)
ax[0].set_xticks([])
ax[0].xaxis.set_tick_params(width=0)
ax[0].set_ylim(np.min(y_ndvi_mean) - 0.003, np.max(y_ndvi_mean) + 0.003)
ax[0].set_xlim(1984, 2014)
ax[0].vlines(
    most_common_years,
    ax[0].get_ylim()[0],
    ax[0].get_ylim()[1],
    color="black",
    linestyle="--",
)

In [ ]:
fig, ax = plt.subplots(
    3, 1, figsize=(6, 8), gridspec_kw={"wspace": 0.1, "hspace": 0.05}
)
ax[0].plot(t, y_ndvi_mean, "-*", color="green", alpha=0.7, label="EF")
ax[0].fill_between(t, ci_ndvi_0_pd, ci_ndvi_1_pd, color="green", alpha=0.2)
# ax[0].set_ylabel("NDVI", fontsize=14)
ax[0].tick_params(axis="y", colors="green")
ax[0].tick_params(axis="both", which="major", labelsize=14)
ax[0].set_xticks([])
ax[0].xaxis.set_tick_params(width=0)
ax[0].set_ylim(np.min(y_ndvi_mean) - 0.03, np.max(y_ndvi_mean) + 0.03)
ax[0].set_xlim(1984, 2014)
ax[0].vlines(
    most_common_years,
    ax[0].get_ylim()[0],
    ax[0].get_ylim()[1],
    color="black",
    linestyle="--",
)

ax[1].plot(t, slp_ndvi_pd, color="green", linewidth=2, label="slope")
ax[1].fill_between(
    t, slp_ndvi_pd - slp_ndvi_sd, slp_ndvi_pd + slp_ndvi_sd, color="green", alpha=0.2
)
ax[1].set_ylim(-0.03, 0.02)
ax[1].set_xticks([])
ax[1].set_xlim(1984, 2014)
# ax[1].set_ylabel("Slope", fontsize=14)ndvi
ax[1].tick_params(axis="y", colors="green")
ax[1].vlines(
    most_common_years,
    ax[1].get_ylim()[0],
    ax[1].get_ylim()[1],
    color="black",
    linestyle="--",
)
ax[1].axhline(0, color="black", linestyle="--", linewidth=1, alpha=0.5)
ax[1].tick_params(axis="y", labelsize=12)

ax02 = ax[1].twinx()
ax02.fill_between(t, 0, slp_pos_mean, color="red", alpha=0.5)
ax02.set_ylim(0.15, 1)
ax02.tick_params(axis="y", colors="red", labelsize=12)
# ax02.set_ylabel("Prob of positive\nslope", fontsize=14)


bin_centers = t[:-1] + np.diff(t) / 2
bin_edges = np.concatenate(
    ([t[0] - (t[1] - t[0]) / 2], bin_centers, [t[-1] + (t[1] - t[0]) / 2])
)
ax[2].hist(cp_ndvi, bins=bin_edges, color="black", alpha=0.5)
# ax[2].set_ylabel("Frequency of\ndetected CPs", fontsize=14, color="black")
ax[2].set_xlim(1984, 2014)
ax[2].vlines(
    most_common_years,
    ax[2].get_ylim()[0],
    ax[2].get_ylim()[1],
    color="black",
    linestyle="--",
)
ax[2].tick_params(axis="both", labelsize=12)

for a in ax:
    for label in a.get_xticklabels():
        label.set_fontproperties(font_manager.FontProperties(weight="bold"))
    for label in a.get_yticklabels():
        label.set_fontproperties(font_manager.FontProperties(weight="bold"))

for a in [ax02]:
    for label in a.get_xticklabels():
        label.set_fontproperties(font_manager.FontProperties(weight="bold"))
    for label in a.get_yticklabels():
        label.set_fontproperties(font_manager.FontProperties(weight="bold"))


fig.align_ylabels()
# plt.savefig(
#     "../manuscript/figures/NDVI_dynamics.png", bbox_inches="tight", pad_inches=0.1
# )

# Causal Inference

### Our goal is to find the impact of 1994 land cover change on NDVI


#### Find years where rbeast found a CP in EF


In [ ]:
ef_changed_cp = ndvi.isel(time=0).copy(data=ef_changed_rbeast.trend.cp[0, :, :])
ef_changed_cpPr = ndvi.isel(time=0).copy(data=ef_changed_rbeast.trend.cpPr[0, :, :])
ef_changed_cp_sig = ef_changed_cp.where(ef_changed_cpPr > 0.5)

#### Find those pixels where CPs happened in 1994, then take the spatial mean of percent covers and NDVI


In [ ]:
names = ["EF", "Shrub", "Herb", "Sparse"]
changed_1994 = ef_changed_cp_sig == 1994
lc_changed_1994 = lc.where(changed_1994 == True)
lc_changed_1994_df = lc_changed_1994.mean(["lat", "lon"]).to_pandas()
lc_changed_1994_df.columns = names
ef_changed_1994 = lc_changed_1994.isel(band=0)
sh_changed_1994 = lc_changed_1994.isel(band=1)
sparse_changed_1994 = lc_changed_1994.isel(band=3)

ef_changed_1994 = lc_changed_1994.isel(band=0)
ef_changed_1994_mean = ef_changed_1994.mean(["lat", "lon"])

sh_changed_1994 = lc_changed_1994.isel(band=1)
sh_changed_1994_mean = sh_changed_1994.mean(["lat", "lon"])

hb_changed_1994 = lc_changed_1994.isel(band=2)
hb_changed_1994_mean = hb_changed_1994.mean(["lat", "lon"])

sparse_changed_1994 = lc_changed_1994.isel(band=3)
sparse_changed_1994_mean = sparse_changed_1994.mean(["lat", "lon"])

ndvi_changed_1994 = ndvi.where(changed_1994 == True)
lc_not_changed_1994 = lc.where(changed_1994 == False)
lc_not_changed_1994_stacked = lc_not_changed_1994.stack(z=["lon", "lat"])

In [ ]:
metadata = rb.args(whichDimIsTime=1, season="none", startTime=1984)
prior = rb.args(trendMinSepDist=1)
mcmc = rb.args(seed=1)
extra = rb.args(  # a set of options to specify the outputs or computational configurations
    dumpInputData=True,  # make a copy of the aggregated input data in the beast ouput
    numThreadsPerCPU=1,  # Paralell  computing: use 2 threads per cpu core
    numParThreads=0,  # `0` means using all CPU cores: total num of ParThreads = numThreadsPerCPU * core Num
    printOptions=False,
    computeTrendSlope=True,
    computeCredible=True,
)
ef_changed_1994_rbeast = rb.beast123(
    ef_changed_1994.values, metadata, prior, mcmc, extra
)
sh_changed_1994_rbeast = rb.beast123(
    sh_changed_1994.values, metadata, prior, mcmc, extra
)
hb_changed_1994_rbeast = rb.beast123(
    hb_changed_1994.values, metadata, prior, mcmc, extra
)
sparse_changed_1994_rbeast = rb.beast123(
    sparse_changed_1994.values, metadata, prior, mcmc, extra
)

In [ ]:
Y_ef_changed_1994 = ef_changed_1994_rbeast.trend.Y.reshape(30, -1)
Y_ef_changed_1994_mean = (
    pd.DataFrame(Y_ef_changed_1994, index=t).dropna(axis=1).mean(axis=1)
)
ci_ef_0_1994 = ef_changed_1994_rbeast.trend.CI[:, 0, :, :]
ci_ef_1_1994 = ef_changed_1994_rbeast.trend.CI[:, 1, :, :]
ci_ef_0_1994_pd = ndvi.copy(data=ci_ef_0_1994).mean(["lat", "lon"])
ci_ef_1_1994_pd = ndvi.copy(data=ci_ef_1_1994).mean(["lat", "lon"])

Y_sh_changed_1994 = sh_changed_1994_rbeast.trend.Y.reshape(30, -1)
Y_sh_changed_1994_mean = (
    pd.DataFrame(Y_sh_changed_1994, index=t).dropna(axis=1).mean(axis=1)
)
ci_sh_0_1994 = sh_changed_1994_rbeast.trend.CI[:, 0, :, :]
ci_sh_1_1994 = sh_changed_1994_rbeast.trend.CI[:, 1, :, :]
ci_sh_0_1994_pd = ndvi.copy(data=ci_sh_0_1994).mean(["lat", "lon"])
ci_sh_1_1994_pd = ndvi.copy(data=ci_sh_1_1994).mean(["lat", "lon"])

Y_hb_changed_1994 = hb_changed_1994_rbeast.trend.Y.reshape(30, -1)
Y_hb_changed_1994_mean = (
    pd.DataFrame(Y_hb_changed_1994, index=t).dropna(axis=1).mean(axis=1)
)
ci_hb_0_1994 = hb_changed_1994_rbeast.trend.CI[:, 0, :, :]
ci_hb_1_1994 = hb_changed_1994_rbeast.trend.CI[:, 1, :, :]
ci_hb_0_1994_pd = ndvi.copy(data=ci_hb_0_1994).mean(["lat", "lon"])
ci_hb_1_1994_pd = ndvi.copy(data=ci_hb_1_1994).mean(["lat", "lon"])

Y_sparse_changed_1994 = sparse_changed_1994_rbeast.trend.Y.reshape(30, -1)
Y_sparse_changed_1994_mean = (
    pd.DataFrame(Y_sparse_changed_1994, index=t).dropna(axis=1).mean(axis=1)
)
ci_sparse_0_1994 = sparse_changed_1994_rbeast.trend.CI[:, 0, :, :]
ci_sparse_1_1994 = sparse_changed_1994_rbeast.trend.CI[:, 1, :, :]
ci_sparse_0_1994_pd = ndvi.copy(data=ci_sparse_0_1994).mean(["lat", "lon"])
ci_sparse_1_1994_pd = ndvi.copy(data=ci_sparse_1_1994).mean(["lat", "lon"])

# Create a dataset that conist of the obserbed NDVIs that changes in


In [ ]:
# Get the indices of pixels within 50 km radius
ndvi_changed_stacked = ndvi_changed_1994.stack(z=["lon", "lat"])
ndvi_not_changed_stacked = ndvi_not_changed.stack(z=["lon", "lat"])

# select a random year and calculate the distances within 50 km radius and get all indices
changed_year_ndvi = ndvi_changed_stacked.isel(time=10)
df = changed_year_ndvi.to_dataframe()
coords = np.radians(df[["lat", "lon"]])
dist = DistanceMetric.get_metric("haversine")
tree = BallTree(coords, metric=dist)
indices = tree.query_radius(coords, r=0.005)

In [ ]:
lc_mean_neighbors = []
data_ndvi = []
arr_idx = []

for k in tqdm(range(len(indices))):
    # for k in tqdm(np.arange(457635, 457640)):
    data = []
    lc_tmp = []

    idx = indices[k]
    # Get the changed NDVI values of the centeral pixel
    center_pixel = idx[np.where(idx == k)]
    center_pixel_ndvi = ndvi_changed_stacked[:, center_pixel].values.squeeze()

    # continue if the central pixel is nan
    if np.isnan(center_pixel_ndvi).all():
        continue
    if np.isnan(center_pixel_ndvi).sum() > 5:
        continue

    data.append(center_pixel_ndvi)

    # Go over the neighboring pixels and get the NDVI values of unchanged pixels
    for i in range(len(idx)):
        if idx[i] == center_pixel:
            continue
        tmp1 = ndvi_not_changed_stacked[:, idx[i]].values.squeeze()
        # skip if the neighboring pixel is all nans
        if np.isnan(tmp1).all():
            continue
        if np.isnan(tmp1).sum() > 5:
            continue
        data.append(tmp1)
    data_ndvi.append(np.array(data).transpose())
    arr_idx.append(k)  # Get the index of changed pixel
    lc_mean_neighbors.append(lc_not_changed_1994_stacked[:, :, idx[i]].values)

In [ ]:
len(data_ndvi)

In [ ]:
with open("../manuscript/outputs/disturbed_ndvi_1994", "wb") as fp:
    pickle.dump(data_ndvi, fp)
with open("../manuscript/outputs/index_disturbed_ndvi_1994", "wb") as fp:
    pickle.dump(arr_idx, fp)

## Now the data for causal inference analyses is ready. We use R to do the analyses. Then import outputs from R to here for further analyses.


In [ ]:
import pyreadr

# Read the X and y values of NDVIs that are used in causal_impact.R2 file for fitting the rbeast and plotting it
rdata = pyreadr.read_r("../manuscript/outputs/ci_results_data.RData")
y_ndvi = rdata["y_ndvi"]
X_ndvi = rdata["X_ndvi"]
data_mean = rdata["data_mean_ndvi"]
ndvi_changed_1994_mean_r = data_mean["y_mean_ndvi"]
ndvi_not_changed_1994_mean_r = data_mean["X_mean_ndvi"]
results_ndvi = rdata["results_ndvi"]
response_ndvi = rdata["response_ndvi"].T
pred_ndvi = rdata["pred_ndvi"].T
point_effect = rdata["point.effect_ndvi"]
point_effect_lower = rdata["point.effect.lower_ndvi"].T
point_effect_upper = rdata["point.effect.upper_ndvi"].T
pred_lower = rdata["pred.lower_ndvi"].T
pred_upper = rdata["pred.upper_ndvi"].T

#### Fit the rbeast for the NDVI values used in causal inference analyses


In [ ]:
metadata = rb.args(whichDimIsTime=1, season="none", startTime=1984)
prior = rb.args(trendMinSepDist=1)
mcmc = rb.args(seed=1)
extra = rb.args(  # a set of options to specify the outputs or computational configurations
    dumpInputData=True,  # make a copy of the aggregated input data in the beast ouput
    numThreadsPerCPU=1,  # Paralell  computing: use 2 threads per cpu core
    numParThreads=0,  # `0` means using all CPU cores: total num of ParThreads = numThreadsPerCPU * core Num
    printOptions=False,
    computeTrendSlope=True,
    computeCredible=True,
)
ndvi_rbeast_X = rb.beast123(X_ndvi, metadata, prior, mcmc, extra)
ndvi_rbeast_y = rb.beast123(y_ndvi, metadata, prior, mcmc, extra)

In [ ]:
y_ndvi_mean_changed_1994 = ndvi_rbeast_y.trend.Y
y_ndvi_mean_changed_1994_mean = pd.DataFrame(y_ndvi_mean_changed_1994, index=t).mean(
    axis=1
)
ci_ndvi_0_pd_changed_1994 = ndvi_rbeast_y.trend.CI[:, 0]
ci_ndvi_1_pd_changed_1994 = ndvi_rbeast_y.trend.CI[:, 1]
ci_ndvi_0_pd_changed_1994 = pd.DataFrame(ci_ndvi_0_pd_changed_1994, index=t).mean(
    axis=1
)
ci_ndvi_1_pd_changed_1994 = pd.DataFrame(ci_ndvi_1_pd_changed_1994, index=t).mean(
    axis=1
)

y_ndvi_mean_not_changed_1994 = ndvi_rbeast_X.trend.Y
y_ndvi_mean_not_changed_1994_mean = pd.DataFrame(
    y_ndvi_mean_not_changed_1994, index=t
).mean(axis=1)
ci_ndvi_0_pd_not_changed_1994 = ndvi_rbeast_X.trend.CI[:, 0]
ci_ndvi_1_pd_not_changed_1994 = ndvi_rbeast_X.trend.CI[:, 1]
ci_ndvi_0_pd_not_changed_1994 = pd.DataFrame(
    ci_ndvi_0_pd_not_changed_1994, index=t
).mean(axis=1)
ci_ndvi_1_pd_not_changed_1994 = pd.DataFrame(
    ci_ndvi_1_pd_not_changed_1994, index=t
).mean(axis=1)

In [ ]:
from scipy import stats

ci_variables = pyreadr.read_r("../manuscript/outputs/ci_aggregated.RData")
pred = ci_variables["pred"].squeeze()
pred_lower = ci_variables["pred.lower"].squeeze()
pred_upper = ci_variables["pred.upper"].squeeze()
response = ci_variables["response"].squeeze()
point_effect = ci_variables["point.effect"].squeeze()
point_effect_lower = ci_variables["point.effect.lower"].squeeze()
point_effect_upper = ci_variables["point.effect.upper"].squeeze()

# Calculate standard error of the mean
y_mean_ndvi = np.nanmean(y_ndvi, axis=1)
y_std_ndvi = np.nanstd(y_ndvi, axis=1)
sem_y = y_std_ndvi / np.sqrt(y_ndvi.shape[0])
y_upper_bound = y_mean_ndvi + sem_y
y_lower_bound = y_mean_ndvi - sem_y

# Everything above ok


In [ ]:
Y_ef = ef_changed_rbeast.trend.Y.reshape(30, -1)
Y_ef_mean = pd.DataFrame(Y_ef, index=t).dropna(axis=1).mean(axis=1)
Y_ef_std = pd.DataFrame(Y_ef, index=t).dropna(axis=1).std(axis=1)
cpOccPr_ef = (
    ef_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_ef = (
    ef_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_ef = (
    ef_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_ef = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_ef)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_ef_0 = ef_changed_rbeast.trend.CI[:, 0, :, :]
ci_ef_1 = ef_changed_rbeast.trend.CI[:, 1, :, :]
ci_ef_0_pd = ndvi.copy(data=ci_ef_0).mean(["lat", "lon"])
ci_ef_1_pd = ndvi.copy(data=ci_ef_1).mean(["lat", "lon"])
most_common_cp_ef = most_common_values(cp_ef.values.squeeze(), 2)

Y_sh = sh_changed_rbeast.trend.Y.reshape(30, -1)
Y_sh_mean = pd.DataFrame(Y_sh, index=t).dropna(axis=1).mean(axis=1)
Y_sh_std = pd.DataFrame(Y_sh, index=t).dropna(axis=1).std(axis=1)
cpOccPr_sh = (
    sh_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_sh = (
    sh_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_sh = (
    sh_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_sh = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_sh)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_sh_0 = sh_changed_rbeast.trend.CI[:, 0, :, :]
ci_sh_1 = sh_changed_rbeast.trend.CI[:, 1, :, :]
ci_sh_0_pd = ndvi.copy(data=ci_sh_0).mean(["lat", "lon"])
ci_sh_1_pd = ndvi.copy(data=ci_sh_1).mean(["lat", "lon"])
most_common_cp_sh = most_common_values(cp_sh.values.squeeze(), 2)

Y_hb = hb_changed_rbeast.trend.Y.reshape(30, -1)
Y_hb_mean = pd.DataFrame(Y_hb, index=t).dropna(axis=1).mean(axis=1)
Y_hb_std = pd.DataFrame(Y_hb, index=t).dropna(axis=1).std(axis=1)
cpOccPr_hb = (
    hb_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_hb = (
    hb_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_hb = (
    hb_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_hb = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_hb)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_hb_0 = hb_changed_rbeast.trend.CI[:, 0, :, :]
ci_hb_1 = hb_changed_rbeast.trend.CI[:, 1, :, :]
ci_hb_0_pd = ndvi.copy(data=ci_hb_0).mean(
    ["lat", "lon"]
)  # Lets increse the size of the figured.cp  # Detected change points with probability more than 50%
ci_hb_1_pd = ndvi.copy(data=ci_hb_1).mean(["lat", "lon"])

Y_sparse = sparse_changed_rbeast.trend.Y.reshape(30, -1)
Y_sparse_mean = pd.DataFrame(Y_sparse, index=t).dropna(axis=1).mean(axis=1)
Y_sparse_std = pd.DataFrame(Y_sparse, index=t).dropna(axis=1).std(axis=1)
cpOccPr_sparse = (
    sparse_changed_rbeast.trend.cpOccPr
)  # Probability of occurance of a change point
cpPr_sparse = (
    sparse_changed_rbeast.trend.cpPr
)  # Probability associated to a detected change point
cp_sparse = (
    sparse_changed_rbeast.trend.cp
)  # Detected change points with probability more than 50%
cp_sparse = pd.DataFrame(
    ndvi.isel(time=range(9))
    .copy(data=cp_sparse)
    .isel(time=0)
    .stack(z=["lat", "lon"])
    .values
).dropna()
ci_sparse_0 = sparse_changed_rbeast.trend.CI[:, 0, :, :]
ci_sparse_1 = sparse_changed_rbeast.trend.CI[:, 1, :, :]
ci_sparse_0_pd = ndvi.copy(data=ci_sparse_0).mean(["lat", "lon"])
ci_sparse_1_pd = ndvi.copy(data=ci_sparse_1).mean(["lat", "lon"])
most_common_cp_sparse = most_common_values(cp_sparse.values.squeeze(), 2)

In [ ]:
index = (results_ndvi.iloc[:, 4] < 0.01).squeeze()
sig_response_ndvi = response_ndvi[index]
sig_pred_ndvi = pred_ndvi[index]
sig_point_effect = point_effect[index]
sig_point_effect_lower = point_effect_lower[index]
sig_point_effect_upper = point_effect_upper[index]
sig_pred_lower = pred_lower[index]
sig_pred_upper = pred_upper[index]

In [ ]:
# Calculate standard error of the mean
sem_response = stats.sem(sig_response_ndvi, axis=0)

# Calculate the means
mean_response = sig_response_ndvi.mean(axis=0)
mean_pred = sig_pred_ndvi.mean(axis=0)

# Calculate the 95th percentile of the confidence interval
upper_percentile = np.percentile(sig_pred_upper, 95, axis=0)
lower_percentile = np.percentile(sig_pred_lower, 5, axis=0)

# Create subplots
fig, ax = plt.subplots(
    figsize=(8, 5),
    gridspec_kw={
        "wspace": 0.1,
        "hspace": 0.05,
    },
)

# Plot observed data with SEM error bars
ax.plot(t, mean_response, color="black", label="Observed")
ax.errorbar(t, mean_response, yerr=sem_response, fmt="none", color="gray", alpha=0.5)

# Plot predicted data with 95th percentile error bars
ax.plot(t, mean_pred, color="red", label="Predicted")
ax.errorbar(
    t,
    mean_pred,
    yerr=[mean_pred - lower_percentile, upper_percentile - mean_pred],
    fmt="none",
    color="red",
    alpha=0.5,
)

# Add legend and labels
ax.legend()
ax.set_xlabel("Time")
ax.set_ylabel("Value")

plt.show()

In [ ]:
from scipy import stats
import numpy as np

# Calculate standard error of the mean
sem_response = stats.sem(sig_response_ndvi, axis=0)

# Calculate the means
mean_response = sig_response_ndvi.mean(axis=0)
mean_pred = sig_pred_ndvi.mean(axis=0)

# Calculate the 95th percentile of the confidence interval
upper_percentile = np.percentile(sig_pred_upper, 95, axis=0)
lower_percentile = np.percentile(sig_pred_lower, 5, axis=0)

# Create subplots
fig, ax = plt.subplots(
    2,
    1,
    figsize=(6, 8),
    gridspec_kw={
        "wspace": 0.1,
        "hspace": 0.05,
    },
)

# Plot observed data with SEM filled in
ax[0].plot(t, mean_response, color="black", label="Observed")
ax[0].fill_between(
    t,
    mean_response - sem_response,
    mean_response + sem_response,
    color="gray",
    alpha=0.5,
)

# Plot predicted data
ax[0].plot(t, mean_pred, color="red", label="Predicted")
ax[0].fill_between(t, lower_percentile, upper_percentile, color="red", alpha=0.5)

In [ ]:
from scipy import stats

# Calculate standard error of the mean
sem_response = stats.sem(sig_response_ndvi, axis=0)

# Calculate the means
mean_response = sig_response_ndvi.mean(axis=0)
mean_pred = sig_pred_ndvi.mean(axis=0)

# Create subplots
fig, ax = plt.subplots(
    2,
    1,
    figsize=(6, 8),
    gridspec_kw={
        "wspace": 0.1,
        "hspace": 0.05,
    },
)

# Plot observed data with SEM filled in
ax[0].plot(t, mean_response, color="black", label="Observed")
ax[0].fill_between(
    t,
    mean_response - sem_response,
    mean_response + sem_response,
    color="gray",
    alpha=0.5,
)

# Plot predicted data
ax[0].plot(t, mean_pred, color="red", label="Predicted")
ax[0].fill_between(
    t,
    sig_pred_lower.mean(axis=0),
    sig_point_effect_upper.mean(axis=0),
    color="red",
    alpha=0.5,
)

In [ ]:
sem_pred_lower

#### Fit Rbeast to this mean rdata and the LCs


In [ ]:
ndvi_changed_1994_mean_rbeast = rb.beast(
    ndvi_changed_1994_mean_r, season="none", start=1984, tseg_minlength=1
)
ndvi_not_changed_1994_mean_rbeast = rb.beast(
    ndvi_not_changed_1994_mean_r, season="none", start=1984, tseg_minlength=1
)

ef_changed_1994_mean_rbeast = rb.beast(
    ef_changed_1994_mean, season="none", start=1984, tseg_minlength=1
)
sh_changed_1994_mean_rbeast = rb.beast(
    sh_changed_1994_mean, season="none", start=1984, tseg_minlength=1
)
hb_changed_1994_mean_rbeast = rb.beast(
    hb_changed_1994_mean, season="none", start=1984, tseg_minlength=1
)
sparse_changed_1994_mean_rbeast = rb.beast(
    sparse_changed_1994_mean, season="none", start=1984, tseg_minlength=1
)

#### Extract the results


In [ ]:
Y_ndvi_changed_1994 = ndvi_changed_1994_mean_rbeast.trend.Y
CI_ndvi_changed_1994 = ndvi_changed_1994_mean_rbeast.trend.CI

Y_ndvi_not_changed_1994 = ndvi_not_changed_1994_mean_rbeast.trend.Y
CI_ndvi_not_changed_1994 = ndvi_not_changed_1994_mean_rbeast.trend.CI
ndvi_not_changed_1994_mean_rbeast.trend.slp

Y_ef_changed_1994 = ef_changed_1994_mean_rbeast.trend.Y
CI_ef_changed_1994 = ef_changed_1994_mean_rbeast.trend.CI

Y_sh_changed_1994 = sh_changed_1994_mean_rbeast.trend.Y
CI_sh_changed_1994 = sh_changed_1994_mean_rbeast.trend.CI

Y_hb_changed_1994 = hb_changed_1994_mean_rbeast.trend.Y
CI_hb_changed_1994 = hb_changed_1994_mean_rbeast.trend.CI

Y_sparse_changed_1994 = sparse_changed_1994_mean_rbeast.trend.Y
CI_sparse_changed_1994 = sparse_changed_1994_mean_rbeast.trend.CI